# Mapping health data in python

In this event we're going to learn how to use a library called geopandas to map health data in Toronto. Geopandas is a powerful tool that allows you to work with spatial information to uncover patterns and develop insights that wouldn't be possible without maps! 



In [ ]:
# install a few libraries we'll need!
!pip install --upgrade geopandas
!pip install mapclassify

### Import libraries that we will use to display maps!

We'll be using `pandas` and `geopandas` today.  These are libraries that allow us to do data science and to do mapping.

In [ ]:
import geopandas as gpd
import pandas as pd
import mapclassify
import matplotlib.pyplot as plt

### Let's get some data!

The data we are going to use come from the Ontario Community Health Profiles project. Let's check it out!

https://www.ontariohealthprofiles.ca/index.php 

We also want to get a 

https://github.com/mwidener/rexdale 

### Load a 'geojson' file into a variable we call "nbrhd"

Once the variable is set, we can then use a function called `.head()` to look at the first few rows.

In [ ]:
nbrhd = gpd.read_file("Toronto_Neighbourhoods.geojson")  #load spatial file into a variable called nbrhd
nbrhd.head() #peak at the top 5 rows in the data we loaded.

In [ ]:
#get geodataframe ready - don't worry too much about this!

important_spat_cols = nbrhd.columns[[4, 5, 17]] #say what columns we like - we want to keep the 5th, 6th, and 18th columns
nbrhd_simple = nbrhd.copy() #make sure we don't overwrite the original data!
nbrhd_simple = nbrhd_simple[important_spat_cols] #get rid of the extra columns
nbrhd_simple['NeighbID'] = nbrhd['AREA_SHORT_CODE'].astype(int) #convert text numbers to real numbers for later
nbrhd_simple.head()

In [ ]:
nbrhd.plot()

### This is great! But it's just a map of Toronto. 

Booooooring. 

We want to bring the health data into this map. How do we do that? 

Well we need to load the file with the health data into python first. 

In [ ]:
toronto_data = pd.read_csv('toronto_health_data_2017.csv')
toronto_data.head()

### OK let's bring the data file together with the spatial file

In [ ]:
nbrhd_simple = nbrhd_simple.merge(toronto_data, on="NeighbID")
nbrhd_simple.head()

Great - everything is together now. Can we map something interesting now? What about asthma?

To do this we can add an argument to the `plot()` function.

We say plot ... AND ... color the neighborhoods with the values stored in the `prev_asthma_0up` column.

In [ ]:
nbrhd_simple.plot(column='prev_asthma_0up')

Pretty!

But ... what does it mean? What's missing? What don't we know? 

We should add some more info... and we can customize things. 

For one - we can change the color scheme by using a `cmap` argument. https://matplotlib.org/stable/tutorials/colors/colormaps.html 

In [ ]:
nbrhd_simple.plot(column='prev_asthma_0up', cmap = 'Oranges')

What about a map legend? 

Map legends tell us what we're looking at on a map. For example, what does a light color mean vs. a dark color? 

In [ ]:
nbrhd_simple.plot(column='prev_asthma_0up', cmap = 'Oranges', legend = True)

We can go event fancier. What if we divide the prevelance of asthma data into four categories - also known as 'quartiles'? 

Geopandas let's you do lots of map classification. 'quantiles' is just one of a few options. 

To do this we need to add even more arguments to the `plot.()` function. In this case, we add `classification = 'quantiles'` and `k=4` to say we want quartiles.

In [ ]:
nbrhd_simple.plot(column='prev_asthma_0up', scheme='quantiles', 
                  k=4, cmap='Oranges',  legend=True)

Where am I getting all this info? You can find information about how a function like `plot()` works in something called an API. It takes a long time to learn all this stuff, but once you get going it only gets easier!

https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html


In [ ]:
fig, axes = plt.subplots(1,1) #tell python we want to have more control over our figure
nbrhd_simple.plot(column='prev_asthma_0up', scheme='quantiles', 
                  k=4, cmap='Oranges', edgecolor='grey', ax=axes, legend=True,
                  legend_kwds={'loc': 4, 'title': 'Percent Asthmatic',
                               'title_fontsize': 10,'fontsize': 8})

Let's try copying and pasting the code in the last cell and changing the column to the mental health prevelance column.

In [ ]:
fig, axes = plt.subplots(1,1) #tell python we want to have more control over our figure
nbrhd_simple.plot(column='prev_mh_20up', scheme='quantiles', 
                  k=4, cmap='Oranges', edgecolor='grey', ax = axes, legend=True,
                  legend_kwds={'loc': 4, 'title': 'Prevelance Mental Health Visits',
                               'title_fontsize': 20,'fontsize': 13})
#we can make the map bigger!
current_fig = plt.gcf()
current_fig.set_size_inches(15,15)

We can also add multiple layers to a map! Let's add a file from Open Data Toronto to this map showing the locations of youth mental health services. https://open.toronto.ca/dataset/wellbeing-youth-health-services/ 

In [ ]:
fig, axes = plt.subplots(1,1) #tell python we want to have more control over our figure
mh_services = gpd.read_file("Health Services.geojson")
mh_services.plot(ax=axes)

How do we combine these maps? Well we tell python to first plot the neighborhood map and THEN we tell it to plot the map of health services points all on the SAME axes.

In [ ]:
fig, axes = plt.subplots(1,1) #tell python we want to have more control over our figure

nbrhd_simple.plot(column='prev_mh_20up', scheme='quantiles', 
                  k=4, cmap='Oranges', edgecolor='grey', ax=axes, legend=True,
                  legend_kwds={'loc': 4, 'title': 'Prevelance Mental Health Visits',
                               'title_fontsize': 20,'fontsize': 13})
mh_services.plot(markersize=20, color='blue', ax = axes)

#we can make the map bigger!
current_fig = plt.gcf()
current_fig.set_size_inches(15,15)

Now you try! Can you make a map of diabetes in 20+ year olds in Toronto? Can you classify your neighbourhoods using 'quantiles' in 5 categories instead of 4? Give it a try!

Can you find another layer from Open Data Toronto to add to your map? Maybe something about bicycling? 